In [26]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
pd.set_option('max_rows', 500)



In [2]:
ls Output/*.csv

Output/speaker_data.csv   Output/str2.csv           Output/str_PNC_IHELP.csv
Output/str.csv            Output/strIHELP.csv


In [3]:
# import data:
data = pd.read_csv('Output/str2.csv')

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (2,3,4,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [6]:
data.head()

,filename,speaker,start,stop,length,word,previous,segment,following,position,cog,maxcog
0,PH00-1-1-JStevens,JStevens,7.502999783,7.623000145,0.120000362,SIX,sp,S,IH1,initial,3846.431559,4383.307129
1,PH00-1-1-JStevens,JStevens,7.782999992,7.992000103,0.209000111,SIX,K,S,sp,final,3145.544096,3660.808105
2,PH00-1-1-JStevens,JStevens,8.211999893,8.312999725,0.100999832,SEVEN,sp,S,EH1,initial,2950.241333,3505.469482
3,PH00-1-1-JStevens,JStevens,24.68799973,24.86700058,0.179000854,SEVENTY,AH1,S,EH1,initial,4608.505811,4747.629883
4,PH00-1-1-JStevens,JStevens,35.41199875,35.45299912,0.041000366,FIRST,ER1,S,T,medial,2083.084717,2083.084717


In [7]:
data.index[0]

0

## Clean

In [8]:
# remove obs don't start with P or I:
tokeep = data.filename.apply(lambda x: x[0]).isin(['I','P'])
data = data[tokeep]

In [9]:
def splitname(col):
    splitted = col.split('-')
    if col[0]=='P':
        return splitted[0]+'-'+splitted[1]+'-'+splitted[2]
    elif col[0]=='I':
        return splitted[0]+'-'+splitted[1]
    
data['subject'] = data.filename.apply(splitname)

def streetfunc(col):
    if 'STREET' in col:
        return 'Yes'
    else:
        return 'No'
data['street'] = data.word.apply(streetfunc)   

# reindex:
data.set_index('subject', drop=False, inplace=True)

In [10]:
data.street.value_counts()

No     204333
Yes      2288
dtype: int64

In [11]:
data = data[data.cog.isnull()==False]

In [12]:
data.drop_duplicates(inplace=True)

In [13]:
# bad entry:
data = data[data.cog != '2PH74-0-1-DDegerberg']
data = data[data.maxcog != '3583.615723PH80-2-1-DGood']

In [14]:
data.cog = data.cog.astype(float)
data.maxcog = data.maxcog.astype(float)

In [15]:
# select 5 or more Str:
data['STR'] = data.segment=='STR'
STRcount = data.groupby('subject').STR.sum()
data['STRcount'] = STRcount

data = data[data.STRcount > 4]

In [16]:
# change vowels

def getFrontness(col):
    if col in set(['AE0','AE1','AE2','EH1','EH0','EH2','EY0','EY1',
                   'EY2','IH0','IH1','IH2','IY1','IY0','IY2']):
        return 'front'
    if col in set(['AA0','AA1','AA2','AH1','AH0','AH2','AO0','AO1','AO2','AW0','AW1','AW2',
                   'AY0','AY1','AY2','ER','OW0','OW1','OW2','OY','OY1','UH1','UW0','UW1','UW2']):
        return 'non-front'
    if col in set(['br','lg','ns']):
        return 'pause'
    else:
        return 'consonant'
    
data['following_frontness'] = data.following.apply(getFrontness)
data.following_frontness.unique()
    
def getHeight(col):
    if col in set(['AE0','AE1','AE2','EH1','EH0','EH2','EY0','EY1',
                   'EY2','IH0','AA0','AA1','AA2','AH1','AH0','AH2','AO0',
                   'AO1','AO2','AW0','AW1','AW2','UH1',
                   'AY0','AY1','AY2','ER','OW0','OW1','OW2','OY','OY1']):
        return 'non-high'
    if col in set(['IH1','IH2','IY1','IY0','IY2','UW0','UW1','UW2']):
        return 'high'
    if col in set(['br','lg','ns']):
        return 'pause'
    else:
        return 'consonant'
    
data['following_height'] = data.following.apply(getHeight)
data.following_height.unique()

def getHF(col):
    if col in set(['AE0','AE1','AE2','EH1','EH0','EH2','EY0','EY1',
                   'EY2','IH0','AA0','AA1','AA2','AH1','AH0','AH2','AO0',
                   'AO1','AO2','AW0','AW1','AW2','UH1',
                   'AY0','AY1','AY2','ER','OW0','OW1','OW2','OY','OY1','UW0','UW1','UW2']):
        return 'not'
    if col in set(['IH1','IH2','IY1','IY0','IY2']):
        return 'highfront'
    if col in set(['br','lg','ns']):
        return 'pause'
    else:
        return 'consonant'
    
data['following_hf'] = data.following.apply(getHF)
data.following_hf.unique()



array(['highfront', 'consonant', 'not', 'pause'], dtype=object)

## Analysis

In [17]:
# create initial data frame:
initial = data[data.position=='initial'].copy()

# normalize data:
data['meancog'] = data.groupby('subject').cog.mean()
data['stdevcog'] = data.groupby('subject').cog.std()
# data['stdevmaxcog'] = data.groupby('subject').maxcog.std()
data['normalizedmean'] = (data.cog - data.meancog)/data.stdevcog
#data['difference'] = data.groupby('subject').normalizedmean

# normalize initial:
initial['meancog'] = initial.groupby('subject').cog.mean()
initial['stdevcog'] = initial.groupby('subject').cog.std()
# initial['stdevmaxcog'] = initial.groupby('subject').maxcog.std()
initial['normalizedmean'] = (initial.cog - initial.meancog)/initial.stdevcog

# create doubli index:
initial.set_index(['subject', 'segment'], drop=False, inplace=True)
data.set_index(['subject', 'segment'], drop=False, inplace=True)

# mean cog values
# data['meancog'] = data.groupby(['subject','segment']).cog.mean()
# data['meanmaxcog'] = data.groupby(['subject','segment']).maxcog.mean()
# data['normalizedmean'] = (data.cog - data.meancog)/data.stdevcog
# data['normalizedmeanmax'] = (data.cog - data.meanmaxcog)/data.stdevmaxcog

# initial['meancog'] = initial.groupby(['subject','segment']).cog.mean()
# initial['meanmaxcog'] = initial.groupby(['subject','segment']).maxcog.mean()
# initial['normalizedmean'] = (initial.cog - initial.meancog)/initial.stdevcog
# initial['normalizedmeanmax'] = (initial.cog - initial.meanmaxcog)/initial.stdevmaxcog

### Speaker Data 

In [18]:
# import data:
speaker_data = pd.read_csv('Output/speaker_data.csv')

speaker_data.replace('?', np.nan, inplace=True)
speaker_data = speaker_data[speaker_data.DOB.isnull()==False]
speaker_data = speaker_data[speaker_data.DOB != '1908']
speaker_data['DOB'] = speaker_data.DOB.astype(float)
speaker_data['Sex'] = speaker_data.Sex.str.lower()

speaker_data.head()

,Subj,Age,Sex,Eth,EdYears,Nbrhood,Location,Name,Year,DOB,Corpus,nasal_bimodality,Phila_bimodality,Income,HS,HS_code
0,PH00-1-2,52,f,o/i,12,Buchanan,South,Nathalie J,2000,1948,PNC,NaN,NaN,NaN,NaN,NaN
1,PH00-1-5,70,m,i,12,Buchanan,South,Bag O,2000,1930,PNC,NaN,NaN,NaN,NaN,NaN
2,PH02-1-1,93,f,w,10,Ellis,South,Ethel,2002,1909,PNC,NaN,NaN,NaN,NaN,NaN
3,PH02-2-4,63,f,r,10,Opal/Prince,Kensington,Anne S,2002,1939,PNC,NaN,NaN,NaN,NaN,NaN
4,PH02-2-5,59,f,r/w,11,Opal/Prince,Kensington,Dorothy C,2002,1943,PNC,NaN,NaN,NaN,NaN,NaN


### Merge

In [19]:
initial = initial.merge(speaker_data, left_on = "subject", right_on = "Subj")
data = data.merge(speaker_data, left_on = "subject", right_on = "Subj")

In [20]:
data.head()

,filename,speaker,start,stop,length,word,previous,segment,following,position,...,Location,Name,Year,DOB,Corpus,nasal_bimodality,Phila_bimodality,Income,HS,HS_code
0,PH00-1-1-JStevens,JStevens,7.502999783,7.623000145,0.120000362,SIX,sp,S,IH1,initial,...,South,John S,2000,1979,PNC,NaN,NaN,NaN,NaN,NaN
1,PH00-1-1-JStevens,JStevens,7.782999992,7.992000103,0.209000111,SIX,K,S,sp,final,...,South,John S,2000,1979,PNC,NaN,NaN,NaN,NaN,NaN
2,PH00-1-1-JStevens,JStevens,8.211999893,8.312999725,0.100999832,SEVEN,sp,S,EH1,initial,...,South,John S,2000,1979,PNC,NaN,NaN,NaN,NaN,NaN
3,PH00-1-1-JStevens,JStevens,24.68799973,24.86700058,0.179000854,SEVENTY,AH1,S,EH1,initial,...,South,John S,2000,1979,PNC,NaN,NaN,NaN,NaN,NaN
4,PH00-1-1-JStevens,JStevens,35.41199875,35.45299912,0.041000366,FIRST,ER1,S,T,medial,...,South,John S,2000,1979,PNC,NaN,NaN,NaN,NaN,NaN


## Write to CSV

In [21]:
data.to_csv("data.csv",index = False)
initial.to_csv("init.csv",index = False)
